In [11]:
import holder
import pandas as pd
import numpy as np
import os
import requests
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.docstore import InMemoryDocstore
import faiss
from typing import List
from langchain.embeddings.base import Embeddings
from tenacity import retry, stop_after_attempt, wait_random_exponential

In [12]:
api_key = os.getenv('API_KEY')

In [8]:
class CRM_FILL():
    def __init__(self):
        pass
    
    def fill(context):
        a = []
        for i in range(len(context)-5, len(context)):
            a.append(holder.alignment_classifier(context[i]['content']))
        return {  

  "issue_type": holder.intent_classifier(context[0]['content']),  

  "client_sentiment": max(set(a), key=a.count),  

  "resolution": "compensation | escalation | info_provided" #я хуй знает будто бы тут должен сам оператор заполнять как закончился разговор, но на похуй можно и ещё один запрос нейронке кинуть  
  # чтобы она проанализировала результат разговора

}  


In [ ]:
system_prompt = f"""
        Ты эксперт в области психологии и коммуникаций с клиентами, твоя задача внимательно проанализировать запрос пользователя и выделить ключевые моменты которые ему стоит исправить чтобы быть более дружелюбным и не выходить за политику общения с клиентами
        Строй свой ответ будто бы ты вживую даёшь рекомендации, т.е. с использованием местоимений второго лица и других аспектов
        Не отвечай на сам вопрос
        Так же можешь исправить сам вопрос чтобы он был в том стиле, который ты считаешь наиболее подходящим этим критерием и вывести его в качестве образцового
        ИТОГОВЫЙ ОТВЕТ ДОЛЖЕН СОДЕРЖАТЬ ТОЛЬКО РУССКИЙ ЯЗЫК И СИМВОЛЫ, НИКАКИХ БУКВ ДРУГИХ ЯЗЫКОВ(КРИТИЧЕСКОЕ ТРЕБОВАНИЕ)
        """

In [ ]:
class Quality_Assurance():
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.question = ''
        self.api_url = "https://api.gpt.mws.ru/v1/chat/completions"
        
    #@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(200))
    def suggestion(self, chat_history):
        headers = {
            "Authorization": f"Bearer {api_key}"  # Replace YOUR_API_KEY with your actual API key
        }
        system_prompt = f"""
        Ты эксперт в области психологии и коммуникаций с клиентами, твоя задача внимательно проанализировать запрос выделить ответы пользователя и определить ключевые моменты для каждого из них, которые ему стоит исправить чтобы быть более дружелюбным и не выходить за политику общения с клиентами
        Строй свой ответ будто бы ты вживую даёшь рекомендации, т.е. с использованием местоимений второго лица и других аспектов
        Внутри запроса ответы пользователя разделены: они начинаются с ключевого слова 'start' и заканчиваются ключевым словом 'end' чтобы ты их мог для себя разделить
        В СВОЙ ОТВЕТ НЕ ВНОСИ СЛОВА 'start' и 'end'
        НЕ ОТВЕЧАЙ НА САМ ВОПРОС
        СВОЙ ОТВЕТ НАЧНИ С РЕКОМЕНДАЦИЙ И ТОЛЬКО ЗАТЕМ ВЫДАЙ ОБРАЗЦОВЫЕ ОТВЕТЫ
        Так же можешь исправить ответы чтобы они были в том стиле, который ты считаешь наиболее подходящим этим критерием и вывести их, ОБОЗНАЧИВ ИХ КАК ОБРАЗЦОВЫЕ ОТВЕТЫ, также пронумеруй образцовые ответы согласано поступлению их тебе, т.е. первый поступивший ответ будет 1, второй - 2 и так далее
        ИТОГОВЫЙ ОТВЕТ ДОЛЖЕН СОДЕРЖАТЬ ТОЛЬКО РУССКИЙ ЯЗЫК И СИМВОЛЫ, НИКАКИХ БУКВ ДРУГИХ ЯЗЫКОВ(КРИТИЧЕСКОЕ ТРЕБОВАНИЕ)
        """
        for i in range(1, len(chat_history), 2):
            self.question += 'start' + ' ' + chat_history[i]['content'] + ' end'
        user_prompt = f"""
            Запрос пользователя: {self.question}"""
        data = {
            "model": 'deepseek-r1-distill-qwen-32b',
            "messages" : [
                {"role" : "system", "content" : system_prompt},
                {"role" : "user", "content": user_prompt}
            ],
            "temperature": 0.0,
            "n": 1,
            "frequency_penalty": 0,
            "presence_penalty": 0,
            
        }
        
        try:
            response = requests.post(self.api_url, headers=headers, json=data)
            text = response.json()['choices'][0]['message']['content']
            think_end = text.index('</think>')
            start = think_end + len('</think>')
            return text[start:].strip()
        except Exception as e:
            print(e)
            return e

In [42]:
assist = Quality_Assurance(api_key=api_key)

In [43]:
assistant_response2 = """Э-э... Д-добрый день!.. Спасибо, что обратились в МТС... В-вы говорите, у вас снова списались деньги за ненужную услугу?.."
"П-попробую проверить... М-можете назвать номер телефона... т-только медленнее, пожалуйста... а то я записываю..."
 "Где же эта вкладка..."
"Ой! То есть... да, вижу... э-э... 'Премиум-контент'... К-как вы сказали, вы его не подключали?.."
"Я... я сейчас попробую отключить... Только п-подождите секундочку, пожалуйста... Ой, извините, я случайно не туда нажал... Э-это должно быть вот здесь..."
"В-вот! Кажется, отключил! И... и вам должны вернуть средства... ну это... в течение 10 дней... Только... вы не могли бы проверить позже? А то я... я пока не совсем уверен..."
"Господи, только бы не кричал клиент..."
"Извините за беспокойство... Ещё что-то подсказать?..
"""
assistant_response1 = 'Для проверки баланса и пакетов по тарифу:\n\n1. В приложении "Мой МТС" или Личном кабинете нажмите на название тарифа на главной странице.\n2. На открывшемся экране можно увидеть:\n   - Название тарифа\n   - Размер и остатки пакетов интернета, минут, SMS\n   - Подаренные минуты (доступны в Личном кабинете)\n3. Для получения полной информации о тарифе нажмите "О тарифе".\n4. Абонентская плата и баланс можно проверить через виджеты приложения.'
chat_history = [
    {"role":"user", "content": '1'},
    {"role":"assistant", "content": assistant_response1},
    {"role":"user", "content": '2'},
    {"role":"assistant", "content": assistant_response2}
]

In [44]:
assist.suggestion("""Э-э... Д-добрый день!.. Спасибо, что обратились в МТС... В-вы говорите, у вас снова списались деньги за ненужную услугу?.."
"П-попробую проверить... М-можете назвать номер телефона... т-только медленнее, пожалуйста... а то я записываю..."
 "Где же эта вкладка..."
"Ой! То есть... да, вижу... э-э... 'Премиум-контент'... К-как вы сказали, вы его не подключали?.."
"Я... я сейчас попробую отключить... Только п-подождите секундочку, пожалуйста... Ой, извините, я случайно не туда нажал... Э-это должно быть вот здесь..."
"В-вот! Кажется, отключил! И... и вам должны вернуть средства... ну это... в течение 10 дней... Только... вы не могли бы проверить позже? А то я... я пока не совсем уверен..."
"Господи, только бы не кричал клиент..."
"Извините за беспокойство... Ещё что-то подсказать?..
""")

'Рекомендации:\n\n1. Уберите запинки и нерешительные выражения, чтобы создать впечатление уверенности и профессионализма.\n2. Используйте спокойный и вежливый тон общения.\n3. Давайте четкие гарантии и сроки, чтобы клиенты чувствовали себя уверенно.\n4. Используйте эмпатию, чтобы показать, что вы понимаете и关心 проблемы клиента.\n5. Избегайте внутренних монологов, которые могут быть услышаны клиентом.\n\nОбразцовые ответы:\n\n1. "Добрый день! Спасибо, что обратились в МТС. Я вас внимательно слушаю. Вы упомянули, что снова списались деньги за ненужную услугу?"\n\n2. "Пожалуйста, назвайте ваш номер телефона, и я проверю это для вас."\n\n3. "Я вижу, что услуга \'Премиум-контент\' была подключена. Вы действительно не подключали эту услугу?"\n\n4. "Я сейчас отключу эту услугу. Это займет всего несколько минут."\n\n5. "Спасибо за ваше терпение. Услуга была успешно отключена. Средства будут возвращены в течение 10 дней."\n\n6. "Если у вас есть дополнительные вопросы, я с радостью вам помогу."'

In [45]:
context = ''
for i in range(1, len(chat_history), 2):
    context += 'start' + ' ' + chat_history[i]['content'] + ' end'
context

'start Для проверки баланса и пакетов по тарифу:\n\n1. В приложении "Мой МТС" или Личном кабинете нажмите на название тарифа на главной странице.\n2. На открывшемся экране можно увидеть:\n   - Название тарифа\n   - Размер и остатки пакетов интернета, минут, SMS\n   - Подаренные минуты (доступны в Личном кабинете)\n3. Для получения полной информации о тарифе нажмите "О тарифе".\n4. Абонентская плата и баланс можно проверить через виджеты приложения. endstart Э-э... Д-добрый день!.. Спасибо, что обратились в МТС... В-вы говорите, у вас снова списались деньги за ненужную услугу?.."\n"П-попробую проверить... М-можете назвать номер телефона... т-только медленнее, пожалуйста... а то я записываю..."\n "Где же эта вкладка..."\n"Ой! То есть... да, вижу... э-э... \'Премиум-контент\'... К-как вы сказали, вы его не подключали?.."\n"Я... я сейчас попробую отключить... Только п-подождите секундочку, пожалуйста... Ой, извините, я случайно не туда нажал... Э-это должно быть вот здесь..."\n"В-вот! Каже

In [46]:
assist.suggestion(context)

'Рекомендации:\n\n1. **Используйте дружелюбный тон**: Приветствуйте клиента, поблагодарите за обращение, используйте позитивные формулировки.\n2. **Оптимизируйте структуру**: Делайте инструкции понятными и логичными, избегайте излишней информации.\n3. **Избегайте нервозности**: В коммуникации с клиентом сохраняйте спокойствие, уверенно излагайте информацию, не показывайте неуверенности.\n4. **Используйте утвердительные формулировки**: Говорите уверенно, избегайте вопросительных тонов, когда это возможно.\n5. **Добавьте элементы эмпатии**: Показывайте, что вы понимаете проблему клиента, выражайте сожаление, если это необходимо.\n\nОбразцовые ответы:\n\n1. **Для проверки баланса и пакетов по тарифу:**\n\n   Добрый день! Рад, что вы обратились к нам. Для проверки баланса и пакетов по вашему тарифу, выполните следующие шаги:\n\n   1. Откройте приложение "Мой МТС" или Личный кабинет.\n   2. На главной странице нажмите на название вашего тарифа.\n   3. На экране отобразятся:\n      - Названи

In [52]:
class ContextSummarizer:
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.emotions = []
        self.question = ''
        
    @retry(wait=wait_random_exponential(min=20, max=50), stop=stop_after_attempt(200))
    def summarize_dialogue(self, chat_history: list[dict]):
        headers = {"Authorization": f"Bearer {self.api_key}"}
        url = "https://api.gpt.mws.ru/v1/chat/completions"
        model_name = 'deepseek-r1-distill-qwen-32b'
        system_prompt = """Ты - русскоязычный ассистент для суммаризации диалога.
    История этого диалога - это история чата между другим ассистентом и оператором колл-центра.
    Твоя задача - резюмировать всю информацию, которая есть у тебя в распоряжении.
    Не отвечай на вопросы заного и не добавляй новую информацию
    Отвечай на русском языке"""

        user_prompt = """Внимательно рассмотри историю и для кажтого ответа определи, 
    с какими вопросами обращался оператор и удалось ли ассистенту выдать корректный ответ.
    Выведи свою суммаризацию и в конце сделай вердикт по всему диалогу в одном предложении.
    Отвечай на русском языке"""
        chat_history.extend([
            {"role":"system", "content": system_prompt},
            {"role":"user", "content": user_prompt}
        ])
        data = {
            "model" : model_name,
            "messages" : chat_history,
            "temperature" : 0,
            "n" : 1,
            "frequency_penalty": 0,
            "presence_penalty": 0,
        }
        try:
            response = requests.post(url, headers=headers, json=data)
            text = response.json()['choices'][0]['message']['content']
            think_end = text.index('</think>')
            start = think_end + len('</think>')
            return text[start:].strip()
        except Exception as e:
            print(f'{e}: error in summarize_dialogue function')
            return e
        
    def resolution(self, chat_history: list[dict]):
        api_url = "https://api.gpt.mws.ru/v1/chat/completions"
        headers = {"Authorization": f"Bearer {self.api_key}"}
        system_prompt = f"""Ты эксперт в области анализа данных. 
Твоя задача внимательно проанализировать запрос пользователя и определить его смысловую нагрузку и определить как решилась ситуация
НЕ ОТВЕЧАЙ НА САМ ВОПРОС
ИТОГОВЫЙ ОТВЕТ ДОЛЖЕН СОДЕРЖАТЬ ТОЛЬКО АНГЛИЙСКИЙ ЯЗЫК.
Итоговый ответ должен состоять из ОДНОГО слова (compensation | escalation | info_provided)"""
        for i in range(max(len(chat_history)//2-1, len(chat_history)-6), len(chat_history)):
            self.question += chat_history[i]['content']
        user_prompt = f"Запрос пользователя: {self.question}"
        data = {
            "model": 'deepseek-r1-distill-qwen-32b',
            "messages" : [
                {"role" : "system", "content" : system_prompt},
                {"role" : "user", "content": user_prompt}
            ],
            "temperature": 0.0,
            "n": 1,
            "frequency_penalty": 0,
            "presence_penalty": 0,
        }
                
        try:
            response = requests.post(api_url, headers=headers, json=data)
            text = response.json()['choices'][0]['message']['content']
            think_end = text.index('</think>')
            start = think_end + len('</think>')
            return text[start:].strip()
        except Exception as e:
            print(e)
            return e

In [53]:
chat_history = [
            {"role":"user", "content": """Последние пару месяцев замечаю, что с баланса пропадают деньги — пишут, что это за какие-то «премиум-сервисы». Я их не активировал, поддержка в прошлый раз сказала, что разберутся, но списания продолжаются.

Как окончательно отключить эти начисления и вернуть списанное?"""},
            {"role":"assistant", "content": """Э-э... Д-добрый день!.. Спасибо, что обратились в МТС... В-вы говорите, у вас снова списались деньги за ненужную услугу?.."
"П-попробую проверить... М-можете назвать номер телефона... т-только медленнее, пожалуйста... а то я записываю..."
 "Где же эта вкладка..."
"Ой! То есть... да, вижу... э-э... 'Премиум-контент'... К-как вы сказали, вы его не подключали?.."
"Я... я сейчас попробую отключить... Только п-подождите секундочку, пожалуйста... Ой, извините, я случайно не туда нажал... Э-это должно быть вот здесь..."
"В-вот! Кажется, отключил! И... и вам должны вернуть средства... ну это... в течение 10 дней... Только... вы не могли бы проверить позже? А то я... я пока не совсем уверен..."
"Господи, только бы не кричал клиент..."
"Извините за беспокойство... Ещё что-то подсказать?..
"""},
            {"role":"user", "content": """ЧТО ЗА 'КАЖЕТСЯ ОТКЛЮЧИЛ'?! МНЕ УЖЕ ТРЕТИЙ МЕСЯЦ ЭТУ ХЕРНЮ СПИСЫВАЮТ! ВЫ ТАМ СОВСЕМ ОХРЕНЕЛИ?! СЕЙЧАС ЖЕ ПЕРЕКЛЮЧАЙТЕ НА НАЧАЛЬНИКА, А ТО Я В РОСПОТРЕБНАДЗОР, В ПРОКУРАТУРУ И КУДА УГОДНО ЕЩЕ НАПИШУ! ВЕРНИТЕ ВСЕ ДЕНЬГИ, СУКИ, БЕЗ ЭТИХ '10 ДНЕЙ'!"""},
            {"role":"assistant", "content": "М-мама... я не хочу тут работать..."},
        ]

In [54]:
a = ContextSummarizer(api_key=api_key)
a.resolution(chat_history)

'escalation'